# Histogram Mode Detector

## TL;DR - Get normalized sample images averaged along the rotation angle.

In [ ]:
from ess.imaging.data import get_ymir_images_path
from ess.imaging.io import FilePath
from ess.imaging.normalize import NormalizedSampleImages
from ess.imaging.workflow import (
    YmirImageNormalizationWorkflow,
    ImageDetectorName,
    RotationMotionSensorName,
)

wf = YmirImageNormalizationWorkflow()
wf[FilePath] = (
    get_ymir_images_path()
)  # Replace with the path to your images in nexus file.
wf[ImageDetectorName] = 'orca'
wf[RotationMotionSensorName] = 'motion_cabinet_2'
normalized = wf.compute(NormalizedSampleImages)
normalized

In [ ]:
normalized['time', 0].plot(
    title=f'Normalized Sample Image at Rotation Angle of {normalized.coords["rotation_angle"].values[0]:.3f}'
)

You can use a slicer to browse images along the rotation angle.

```python
%matplotlib widget
from plopp import slicer

slicer(normalized)
```

If you want to bin the images by linear steps of angles,
you can use ``sc.groupby`` like below.

```python
%matplotlib widget
import scipp as sc
from plopp import slicer

slicer(normalized.groupby(
    'rotation_angle',
    bins=sc.linspace(dim='rotation_angle', start=-0.5, stop=4.5, num=5, unit='deg'),
).mean('rotation_angle'))
```

## IO

I/O providers can load white beam images and slice them according to the image key.

Note that not like the ``rotation_angle``, ``image_key`` is not assigned as a coordinate.

Instead, we slice the data according to the ``image_key`` and separate image stacks earlier due to performance limitation of 3-dimensional data binning.

> For example, for 400 images of 2_048 x 2_048 pixels couldn't be done in the regular laptop.

### Load the data as a stack of images

In [ ]:
from ess.imaging.data import get_ymir_images_path
from ess.imaging.io import (
    FilePath,
    SampleImageStacksWithLogs,
    RawSampleImageStacks,
    OpenBeamImageStacks,
    DarkCurrentImageStacks,
    AllImageStacks,
)
from ess.imaging.workflow import (
    YmirImageNormalizationWorkflow,
    ImageDetectorName,
    RotationMotionSensorName,
)

wf = YmirImageNormalizationWorkflow()
wf[FilePath] = get_ymir_images_path()
wf[ImageDetectorName] = 'orca'
wf[RotationMotionSensorName] = 'motion_cabinet_2'

In [ ]:
image_stack_types = (
    AllImageStacks,
    RawSampleImageStacks,
    OpenBeamImageStacks,
    DarkCurrentImageStacks,
    SampleImageStacksWithLogs,
)
wf.visualize(image_stack_types)

In [ ]:
results = wf.compute(image_stack_types)
results[SampleImageStacksWithLogs]

### Save Individual Frame

In [ ]:
from ess.imaging.io import export_image_stacks_as_tiff, ImageKey

from pathlib import Path
from tqdm import tqdm


output_dir = Path("images")
export_image_stacks_as_tiff(
    output_dir=output_dir,
    image_stacks=results[AllImageStacks],
    merge_image_by_key=False,
    overwrite=True,
    progress_wrapper=tqdm,
    image_prefix_map={
        ImageKey.SAMPLE: "sample",
        ImageKey.DARK_CURRENT: "dc",
        ImageKey.OPEN_BEAM: "ob",
    },
)

In [ ]:
sorted(output_dir.iterdir())

### Save All Frames in One File

In [ ]:
from ess.imaging.io import export_image_stacks_as_tiff, ImageKey
from tqdm import tqdm
from pathlib import Path


output_dir = Path("images")
export_image_stacks_as_tiff(
    output_dir=output_dir,
    image_stacks=results[AllImageStacks],
    merge_image_by_key=True,
    overwrite=True,
    progress_wrapper=tqdm,
    image_prefix_map={
        ImageKey.SAMPLE: "sample",
        ImageKey.DARK_CURRENT: "dc",
        ImageKey.OPEN_BEAM: "ob",
    },
)

In [ ]:
tuple(output_dir.iterdir())

In [ ]:
!rm -rf images

## Normalization

See :attr:ess.imaging.normalize.normalize_sample_images for more details.

In [ ]:
from ess.imaging.normalize import NormalizedSampleImages

wf[OpenBeamImageStacks] = results[OpenBeamImageStacks]
wf[DarkCurrentImageStacks] = results[DarkCurrentImageStacks]
wf[SampleImageStacksWithLogs] = results[SampleImageStacksWithLogs]
wf.visualize(NormalizedSampleImages)

In [ ]:
wf.compute(NormalizedSampleImages)